# **(Optional)**

In [ ]:
#@markdown # Drop Speakers from Model (Optional)
#@markdown ___
#@markdown <font size="-1.5"> Use this to drop speakers from your model for distribution. You will need to do it for both acoustic and variance models.

drop_model_path = '' #@param {type: "string"}
#@markdown <font size="-1.5"> Type the ID of speakers you'd like to KEEP separated by commas. Ex: "0,3,4" <br>
#@markdown <font size="-1.5"> Note: You can find the ID of speakers in the model by opening the ```spk_map.json``` file in the model folder.<br>
#@markdown <font size="-1.5"> If you see ```{"natural": 0, "power": 1, "silly": 2}``` but only want to keep "natural" and "power", type ```0,1``` below.
retain_speakers = '' #@param {type: "string"}
#@markdown <font size="-1.5"> If you don't know what this means, don't change it.
fill_embed = 'zeros' #@param ['zeros', 'random', 'mean', 'cyclic']

drop_out_path = drop_model_path[:-5] + '_spk-dropped.ckpt'

!python /workspace/content/DiffSinger/scripts/drop_spk.py {drop_model_path} {drop_out_path} --retain {retain_speakers} --fill {fill_embed}




# **Export ONNX**

In [24]:
# Export ONNX
%cd /workspace/content
# from IPython.display import clear_output
# clear_output()
import os
import zipfile
import shutil

#move this here so users doesnt have to wait for this to install for training
print("installing dependencies for onnx conversion...")
!pip install torch==1.13.0 torchvision==0.14.0 torchaudio==0.13.0 -q -q -q 2>/dev/null

# to counter IF the user is to re-run this cell <3
if os.path.exists("/workspace/content/OU_compatible_files"):
    shutil.rmtree("/workspace/content/OU_compatible_files")
    os.remove("/workspace/content/jpn_dict.txt")
else:
    pass

#@markdown <font size="-1.5"> select this if you don't want to see the onnx converter's output
no_output = False # @param {type:"boolean"}

#@markdown <font size="-1.5"> path to your **ACOUSTIC CHECKPOINT** (leave blank if you don't have any): automatically use latest checkpoint that is in the same folder
acoustic_checkpoint_path = "/workspace/content/DiffSinger/MyModel_Acoustic/model_ckpt_steps_54000.ckpt" #@param{type:"string'}
acoustic_folder_name = os.path.basename(os.path.dirname(acoustic_checkpoint_path)) + "_acoustic"
acoustic_folder_path = os.path.dirname(acoustic_checkpoint_path)
print("acoustic_folder_path", acoustic_folder_path)

#@markdown <font size="-1.5"> path to your **VARIANCE CHECKPOINT** (leave blank if you don't have any): automatically use latest checkpoint that is in the same folder
variance_checkpoint_path = "/workspace/content/DiffSinger/MyModel_Variance/model_ckpt_steps_64000.ckpt" #@param{type:"string'}
variance_folder_name = os.path.basename(os.path.dirname(variance_checkpoint_path)) + "_variance"
variance_folder_path = os.path.dirname(variance_checkpoint_path)

#@markdown <font size="-1.5"> path to where you want to save your ONNX files (it will create a folder named "onnx" in this path)
exp_folder = "/workspace/content/DiffSinger/MyModel_export" #@param{type:"string"}

acoustic_onnx_exp = exp_folder + "/onnx/acoustic"
variance_onnx_exp = exp_folder + "/onnx/variance"

if not acoustic_checkpoint_path:
    print("\n")
    print("acoustic ckeckpoint path not specified, not exporting acoustic ONNX...")
else:
    print("\n")
    print("converting acoustic to onnx...")
    #cp stuff cus apparently exporter doesnt work without it
    !cp {acoustic_folder_path}/config.yaml -r /workspace/content/DiffSinger/checkpoints/{acoustic_folder_name}
    search_text = "        args_work_dir = os.path.join("
    replacement = f"        args_work_dir = '{acoustic_folder_path}'"
    with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text in line:
            lines[i] = replacement + "\n"
            break
    with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)
    #incase if anyone wanna change it lmao
    search_text_alt = "        args_work_dir = '"
    replacement_alt = f"        args_work_dir = '{acoustic_folder_path}'"
    with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text_alt in line:
            lines[i] = replacement_alt + "\n"
            break
    with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)

    acoustic_folder_name = "/workspace/content/DiffSinger/" + acoustic_folder_name
    
    if no_output:
        !python /workspace/content/DiffSinger/scripts/export.py acoustic --exp {acoustic_folder_name.replace("_acoustic", "")} --out {exp_folder}/onnx/acoustic >/dev/null 2>&1
    else:
        !python /workspace/content/DiffSinger/scripts/export.py acoustic --exp {acoustic_folder_name.replace("_acoustic", "")} --out {exp_folder}/onnx/acoustic


if not variance_checkpoint_path:
    print("\n")
    print("variance ckeckpoint path not specified, not exporting variance ONNX...")
else:
    print("\n")
    print("converting variance to onnx...")
    #cp stuff cus apparently exporter doesnt work without it
    !cp {variance_folder_path}/config.yaml -r /workspace/content/DiffSinger/checkpoints/{variance_folder_name}
    search_text = "        args_work_dir = os.path.join("
    replacement = f"        args_work_dir = '{variance_folder_path}'"
    with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text in line:
            lines[i] = replacement + "\n"
            break
    with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)
    #incase if anyone wanna change it lmao
    search_text_alt = "        args_work_dir = '"
    replacement_alt = f"        args_work_dir = '{variance_folder_path}'"
    with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text_alt in line:
            lines[i] = replacement_alt + "\n"
            break
    with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)

    variance_folder_name = "/workspace/content/DiffSinger/" + variance_folder_name
    
    if no_output:
        !python /workspace/content/DiffSinger/scripts/export.py variance --exp {variance_folder_name.replace("_variance", "")} --out {exp_folder}/onnx/variance >/dev/null 2>&1
    else:
        !python /workspace/content/DiffSinger/scripts/export.py variance --exp {variance_folder_name.replace("_variance", "")} --out {exp_folder}/onnx/variance


!mv /workspace/content/DiffSinger/MyModel_Acoustic.onnx {acoustic_onnx_exp}/acoustic.onnx
!mv /workspace/content/DiffSinger/MyModel_Acoustic.phonemes.txt {acoustic_onnx_exp}/phonemes.txt

!mv /workspace/content/DiffSinger/MyModel_Variance.dur.onnx {variance_onnx_exp}/dur.onnx
!mv /workspace/content/DiffSinger/MyModel_Variance.linguistic.onnx {variance_onnx_exp}/linguistic.onnx
!mv /workspace/content/DiffSinger/MyModel_Variance.phonemes.txt {variance_onnx_exp}/phonemes.txt
!mv /workspace/content/DiffSinger/MyModel_Variance.pitch.onnx {variance_onnx_exp}/pitch.onnx
!mv /workspace/content/DiffSinger/MyModel_Variance.variance.onnx {variance_onnx_exp}/variance.onnx

## No effect, so use mv code above
# if not variance_checkpoint_path:
#     folder_paths = [acoustic_onnx_exp]
# elif not acoustic_checkpoint_path:
#     folder_paths = [variance_onnx_exp]
# else:
#     folder_paths = [acoustic_onnx_exp, variance_onnx_exp]

# patterns = {"acoustic.onnx": "acoustic.onnx", "dur.onnx": "dur.onnx", "linguistic.onnx": "linguistic.onnx", "pitch.onnx": "pitch.onnx", "variance.onnx": "variance.onnx", "phonemes.txt": "phonemes.txt"}

# for folder_path in folder_paths:
#     for filename in os.listdir(folder_path):
#         for pattern, new_name in patterns.items():
#             if pattern in filename:
#                 old_path = os.path.join(folder_path, filename)
#                 new_path = os.path.join(folder_path, new_name)
#                 if os.path.exists(old_path):
#                     os.rename(old_path, new_path)
# for folder_path in folder_paths:
#     for filename in os.listdir(folder_path):
#         if "acoustic_acoustic." in filename:
#             new_filename = filename.replace("acoustic_acoustic.", "acoustic_")
#         elif "variance_variance." in filename:
#             new_filename = filename.replace("variance_variance.", "variance_")
#         else:
#             new_filename = filename
#         old_path = os.path.join(folder_path, filename)
#         new_path = os.path.join(folder_path, new_filename)
#         os.rename(old_path, new_path)

print("\n")
print("ONNX export complete! Please refer to https://github.com/xunmengshe/OpenUtau/wiki/Voicebank-Development to make your model OU compatible")
print("\n")
print("Or use the 'Build OpenUtau VB' cell to have things set up for you")


/workspace/content
installing dependencies for onnx conversion...
acoustic_folder_path /workspace/content/DiffSinger/MyModel_Acoustic


converting acoustic to onnx...
| found ckpt by name: /workspace/content/DiffSinger/MyModel_Acoustic
| Hparams chains:  []
| Hparams: 
K_step: 400, K_step_infer: 400, accumulate_grad_batches: 1, audio_num_mel_bins: 128, audio_sample_rate: 44100, 
augmentation_args: {'fixed_pitch_shifting': {'enabled': False, 'scale': 0.5, 'targets': [-5.0, 5.0]}, 'random_pitch_shifting': {'enabled': True, 'range': [-5.0, 5.0], 'scale': 0.75}, 'random_time_stretching': {'enabled': True, 'range': [0.5, 2.0], 'scale': 0.75}}, base_config: [], binarization_args: {'num_workers': 0, 'shuffle': True}, binarizer_cls: preprocessing.acoustic_binarizer.AcousticBinarizer, binary_data_dir: /workspace/content/DiffSinger/MyModel_Acoustic/binary, 
breathiness_smooth_width: 0.12, clip_grad_norm: 1, dataloader_prefetch_factor: 2, dataset_size_key: lengths, dictionary: dictionaries/custom

# **Build OpenUtau Voicebank**

In [26]:
# IMPORTANT:
# change the name in Model_export folder to acoustic.onnx, phonemes.txt....

# Build OpenUtau VB
#i need to clean this up it seems
#plan: add a build ou section here by inserting onnx paths (or just the folder containing the folders to the onnx files) to build ou
# ill have a config read function too so i dont have to add checkmark of if people train with embeds or shallow diff or not <3
# yes im lazy rawr x3
%cd /workspace/content
import os
import shutil
import yaml

constr_folder = "/workspace/content/OU_voicebank"
if not os.path.exists(constr_folder):
    os.makedirs(constr_folder)
else:
    shutil.rmtree(constr_folder)
    os.makedirs(constr_folder)

# clear_output()

#@markdown <font size="-1.5"> path to your **ACOUSTIC ONNX FOLDER**
acoustic_onnx_folder = "/workspace/content/DiffSinger/MyModel_export/onnx/acoustic" #@param{type:"string'}
#@markdown <font size="-1.5"> path to the config.yaml of acoustic model
acoustic_config = "/workspace/content/DiffSinger/MyModel_Acoustic/config.yaml" #@param{type:"string'}

#@markdown <font size="-1.5"> path to your **VARIANCE ONNX FOLDER**
variance_onnx_folder = "/workspace/content/DiffSinger/MyModel_export/onnx/variance" #@param{type:"string'}
#@markdown <font size="-1.5"> path to the config.yaml of variance model
variance_config = "/workspace/content/DiffSinger/MyModel_Variance/config.yaml" #@param{type:"string'}

#@markdown <font size="-1.5"> path to your word to phoneme dict (leave blank to use default Japanese dict)
dictionary_path = "/workspace/content/DiffSinger/MyModel_Acoustic/dictionary.txt" #@param{type:"string"}

#@markdown <font size="-1.5"> path to the folder you want to save the zip file to
save_path = "/workspace/content/DiffSinger/MyModel_export" #@param{type:"string"}

#@markdown ___

#@markdown ## Character Configuration | character.txt and character.yaml

#@markdown <font size="-1.5"> your character display name| **required**
name = "SongGen_Singer" #@param{type:"string"}
#@markdown <font size="-1.5"> your character logo | **optional**
image = "" #@param{type:"string"}
image_name = os.path.basename(image)
#@markdown <font size="-1.5"> your character portrait | **optional**
portrait = "" #@param{type:"string"}
portrait_name = os.path.basename(portrait)
#@markdown <font size="-1.5"> the author of the voicebank | **optional**
author = "" #@param{type:"string"}
#@markdown <font size="-1.5"> the voice provider of the voicebank | **optional**
voice = "" #@param{type:"string"}
#@markdown <font size="-1.5"> link to the character's website | **optional**
web = "" #@param{type:"string"}

print("copying files...")
main_stuff = f"{constr_folder}/{name}"
if not os.path.exists(main_stuff):
    os.makedirs(main_stuff)
if not os.path.exists(f"{main_stuff}/dsmain"):
    os.makedirs(f"{main_stuff}/dsmain/embeds/acoustic")
    os.makedirs(f"{main_stuff}/dsmain/embeds/variance")
!cp {acoustic_onnx_folder}/acoustic.onnx {main_stuff}/dsmain
!cp {acoustic_onnx_folder}/phonemes.txt {main_stuff}/dsmain
!cp {acoustic_onnx_folder}/*.emb {main_stuff}/dsmain/embeds/acoustic >/dev/null 2>&1
!cp {variance_onnx_folder}/*.emb {main_stuff}/dsmain/embeds/variance >/dev/null 2>&1
if image:
    !cp {image} {main_stuff}
if portrait:
    !cp {portrait} {main_stiff}

if variance_onnx_folder:
    !cp {variance_onnx_folder}/linguistic.onnx {main_stuff}/dsmain
else:
    pass

print("\n")
print("writing character.txt...")
with open(f"{main_stuff}/character.txt", "w") as file:
    file.write(f"name={name}\n")
    if image:
        file.write(f"image={image_name}\n")
    if author:
        file.write(f"author={author}\n")
    if voice:
        file.write(f"voice={voice}\n")
    if web:
        file.write(f"web={web}\n")

print("\n")
print("writing character.yaml...")
with open(f"{main_stuff}/character.yaml", "w") as file:
    file.write("text_file_encoding: utf-8\n")
    if portrait:
        file.write(f"portrait: {portrait_name}\n")
        file.write("portrait_opacity: 0.45\n")
    file.write("default_phonemizer: OpenUtau.Core.DiffSinger.DiffSingerPhonemizer\n")
    file.write("singer_type: diffsinger\n")
acoustic_emb_files = os.listdir(acoustic_onnx_folder)
acoustic_embeds = []
acoustic_color_suffix = []
for file in acoustic_emb_files:
    if file.endswith(".emb"):
        acoustic_emb = os.path.splitext(file)[0]
        acoustic_embeds.append("dsmain/embeds/acoustic/" + acoustic_emb)
        acoustic_color_suffix.append(acoustic_emb)
subbanks = []
for i, (acoustic_embed_color, acoustic_embed_suffix) in enumerate(zip(acoustic_color_suffix, acoustic_embeds), start=1):
    color = f"{i:02}: {acoustic_embed_color}"
    suffix = f"{acoustic_embed_suffix}"
    subbanks.append({"color": color, "suffix": suffix})
if subbanks:
    with open(f"{main_stuff}/character.yaml", "r") as config:
        i_wanna_die_slash_j = yaml.safe_load(config)
    i_wanna_die_slash_j["subbanks"] = subbanks
    with open(f"{main_stuff}/character.yaml", "w") as config:
        yaml.dump(i_wanna_die_slash_j, config)

print("\n")
print("writing dsconfig.yaml for acoustic...")
with open(f"{main_stuff}/dsconfig.yaml", "w") as file:
    file.write("phonemes: dsmain/phonemes.txt\n")
    file.write("acoustic: dsmain/acoustic.onnx\n")
    file.write("vocoder: nsf_hifigan\n")
    file.write("singer_type: diffsinger\n")
with open(acoustic_config, "r") as config:
    mfking_config = yaml.safe_load(config)
use_energy_embed = mfking_config.get("use_energy_embed")
use_breathiness_embed = mfking_config.get("use_breathiness_embed")
use_shallow_diffusion = mfking_config.get("use_shallow_diffusion")
max_depth = mfking_config.get("K_step")
speakers = mfking_config.get("speakers") #looking back here, why is this even here lmao cus i used acoustic_embeds instead of speakers
augmentation_arg = mfking_config.get("augmentation_args")
pitch_aug = mfking_config.get("use_key_shift_embed")
time_aug = mfking_config.get("use_speed_embed")
voicing = mfking_config.get("use_voicing_embed")
tension = mfking_config.get("use_tension_embed")
with open(f"{main_stuff}/dsconfig.yaml", "r") as config:
    why_are_there_so_many_i_could_prob_make_it_one = yaml.safe_load(config)
why_are_there_so_many_i_could_prob_make_it_one["use_energy_embed"] = use_energy_embed
why_are_there_so_many_i_could_prob_make_it_one["use_breathiness_embed"] = use_breathiness_embed
why_are_there_so_many_i_could_prob_make_it_one["use_shallow_diffusion"] = use_shallow_diffusion
why_are_there_so_many_i_could_prob_make_it_one["max_depth"] = max_depth
why_are_there_so_many_i_could_prob_make_it_one["augmentation_args"] = augmentation_arg
why_are_there_so_many_i_could_prob_make_it_one["use_key_shift_embed"] = pitch_aug
why_are_there_so_many_i_could_prob_make_it_one["use_speed_embed"] = time_aug
why_are_there_so_many_i_could_prob_make_it_one["use_voicing_embed"] = voicing
why_are_there_so_many_i_could_prob_make_it_one["use_tension_embed"] = tension

if subbanks:
    why_are_there_so_many_i_could_prob_make_it_one["speakers"] = acoustic_embeds
with open(f"{main_stuff}/dsconfig.yaml", "w") as config:
    yaml.dump(why_are_there_so_many_i_could_prob_make_it_one, config)


variance_emb_files = os.listdir(variance_onnx_folder)
variance_embeds = []
for file in variance_emb_files:
    if file.endswith(".emb"):
        variance_emb = os.path.splitext(file)[0]
        variance_embeds.append("../dsmain/embeds/variance/" + variance_emb)

print("\n")
print("writing dsdict.yaml...")
if not dictionary_path:
    dict_path = "/workspace/content/jpn_dict.txt"
else:
    dict_path = dictionary_path

# for symbols list
phoneme_dict_path = f"{acoustic_onnx_folder}/dictionary.txt"

dsdict = "dsdict.yaml"

def parse_phonemes(phonemes_str):
    return phonemes_str.split()

entries = []
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG", "cl", "vf"}
vowel_data = []
stop_data = []

# Process the specified dictionary
with open(dict_path, "r") as f:
    for line in f:
        word, phonemes_str = line.strip().split("\t")
        phonemes = parse_phonemes(phonemes_str)
        if len(phonemes) == 1:
            entries.append({"grapheme": word, "phonemes": phonemes})
        else:
            entries.append({"grapheme": word, "phonemes": phonemes})

with open(phoneme_dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        phoneme_type = "vowel" if phoneme[0] in vowel_types else "stop"
        entry = {"symbol": phoneme, "type": phoneme_type}
        if phoneme_type == "vowel":
            vowel_data.append(entry)
        else:
            stop_data.append(entry)

vowel_data.sort(key=lambda x: x["symbol"])
stop_data.sort(key=lambda x: x["symbol"])

dsdict_path = os.path.join(constr_folder, dsdict)
with open(dsdict_path, "w") as f:
    f.write("entries:\n")
    for entry in entries:
        f.write(f"- grapheme: {entry['grapheme']}\n")
        f.write("  phonemes:\n")
        for phoneme in entry["phonemes"]:
            f.write(f"  - {phoneme}\n")

    f.write("\nsymbols:\n")
    for entry in vowel_data + stop_data:
        f.write(f"- symbol: {entry['symbol']}\n")
        f.write(f"  type: {entry['type']}\n")

with open(variance_config, "r") as config:
    mfking_config = yaml.safe_load(config)
sample_rate = mfking_config.get("audio_sample_rate")
hop_size = mfking_config.get("hop_size")
predict_dur = mfking_config.get("predict_dur")
predict_pitch = mfking_config.get("predict_pitch")
use_melody_encoder = mfking_config.get("use_melody_encoder")
predict_voicing = mfking_config.get("predict_voicing")
predict_tension = mfking_config.get("predict_tension")

dur_onnx_path = variance_onnx_folder + "/dur.onnx"
if os.path.exists(dur_onnx_path):
    print("\n")
    print("making dsdur directory and necessary files...")
    os.makedirs(f"{main_stuff}/dsdur")
    !cp {dur_onnx_path} {main_stuff}/dsdur
    !cp {dsdict_path} {main_stuff}/dsdur
    with open(f"{main_stuff}/dsdur/dsconfig.yaml", "w") as file:
        file.write("phonemes: ../dsmain/phonemes.txt\n")
        file.write("linguistic: ../dsmain/linguistic.onnx\n")
        file.write("dur: dur.onnx\n")
    with open(f"{main_stuff}/dsdur/dsconfig.yaml", "r") as config:
        dsdur_config = yaml.safe_load(config)
    dsdur_config["sample_rate"] = sample_rate
    dsdur_config["hop_size"] = hop_size
    dsdur_config["predict_dur"] = predict_dur
    if subbanks:
        dsdur_config["speakers"] = variance_embeds
    with open(f"{main_stuff}/dsdur/dsconfig.yaml", "w") as config:
        yaml.dump(dsdur_config, config)
else:
    print("\n")
    print("dur.onnx not found, skipping on making dsdur folder...")

pitch_onnx_path = variance_onnx_folder + "/pitch.onnx"
if os.path.exists(pitch_onnx_path):
    print("\n")
    print("making dspitch directory and necessary files...")
    os.makedirs(f"{main_stuff}/dspitch")
    !cp {pitch_onnx_path} {main_stuff}/dspitch
    !cp {dsdict_path} {main_stuff}/dspitch
    with open(f"{main_stuff}/dspitch/dsconfig.yaml", "w") as file:
        file.write("phonemes: ../dsmain/phonemes.txt\n")
        file.write("linguistic: ../dsmain/linguistic.onnx\n")
        file.write("pitch: pitch.onnx\n")
        file.write("use_expr: true\n")
    with open(f"{main_stuff}/dspitch/dsconfig.yaml", "r") as config:
        dspitch_config = yaml.safe_load(config)
    dspitch_config["sample_rate"] = sample_rate
    dspitch_config["hop_size"] = hop_size
    dspitch_config["predict_dur"] = predict_pitch
    if subbanks:
        dspitch_config["speakers"] = variance_embeds
    dspitch_config["use_note_rest"] = use_melody_encoder
    with open(f"{main_stuff}/dspitch/dsconfig.yaml", "w") as config:
        yaml.dump(dspitch_config, config)
else:
    print("\n")
    print("pitch.onnx not found, skipping on making dspitch folder...")

variance_onnx_path = variance_onnx_folder + "/variance.onnx"
if os.path.exists(variance_onnx_path):
    print("\n")
    print("making dsvariance directory and necessary files...")
    os.makedirs(f"{main_stuff}/dsvariance")
    !cp {variance_onnx_path} {main_stuff}/dsvariance
    !cp {dsdict_path} {main_stuff}/dsvariance
    with open(f"{main_stuff}/dsvariance/dsconfig.yaml", "w") as file:
        file.write("phonemes: ../dsmain/phonemes.txt\n")
        file.write("linguistic: ../dsmain/linguistic.onnx\n")
        file.write("variance: variance.onnx\n")
    with open(f"{main_stuff}/dsvariance/dsconfig.yaml", "r") as config:
        dsvariance_config = yaml.safe_load(config)
    dsvariance_config["sample_rate"] = sample_rate
    dsvariance_config["hop_size"] = hop_size
    dsvariance_config["predict_dur"] = True #this one will always be true cus if there's no variance model, it shouldnt make this folder in the first place
    dsvariance_config["predict_voicing"] = predict_voicing
    dsvariance_config["predict_tension"] = predict_tension
    if subbanks:
        dsvariance_config["speakers"] = variance_embeds
    with open(f"{main_stuff}/dsvariance/dsconfig.yaml", "w") as config:
        yaml.dump(dsvariance_config, config)
else:
    print("\n")
    print("variance.onnx not found, skipping on making dsvariance folder...")

!rm -rf {dsdict_path}
#im too lazy to write codes so ill just do this, itll only remove those folders if they're empty anyway
!rm -d {main_stuff}/dsmain/embeds/* >/dev/null 2>&1
!rm -d {main_stuff}/dsmain/embeds >/dev/null 2>&1

print("\n")
print("zipping up files...")
!zip -q -9 -r {save_path}/{name}.zip {main_stuff}/*

print("\n")
print("done!")

print("\n")
print("You can download your model zip and use it in OpenUtau! If anything needed to be edit in the config then please do so")

/workspace/content
copying files...


writing character.txt...


writing character.yaml...


writing dsconfig.yaml for acoustic...


writing dsdict.yaml...


making dsdur directory and necessary files...


making dspitch directory and necessary files...


making dsvariance directory and necessary files...


zipping up files...


done!


You can download your model zip and use it in OpenUtau! If anything needed to be edit in the config then please do so
